<a href="https://colab.research.google.com/github/jim-min/gdsc-ai-study/blob/main/week3/3%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GDG 인공지능 기초 3주차 과제**
이번 주차 과제도 퀴즈와 실습으로 구성되어있습니다.

## **<span style="color:green"> [ 퀴즈 ] </span>**

##### 단층 퍼셉트론으로 xor문제를 구현할 수 없다고 이야기 했었는데, 그렇다면 층을 어떻게 구성해야 xor 문제를 표현할 수 있을까요?

① 입력층 - 출력층  
② 입력층 - 은닉층(노드 2개) - 출력층 (활성화 함수 x)  
③ 입력층 - 은닉층(노드 2개) - 출력층 (각 층에 시그모이드 함수)  
④ 입력층 - 은닉층(노드 10개) - 출력층 (각 층에 계단 함수)  

정답과 그 이유를 간단히 작성해주세요!

③\
직관적으로 논리회로에서 XOR 게이트를 만들기 위해선\
두 비트를 각각 AND와 NOR을 거치게 한 후, 두 결과값의 OR을 해야 한다

그렇다면, 은닉층의 두 노드에 각각 AND NOR을 넣고 OR 연산한 걸 출력하면 된다고 생각할 수 있다.

연산이 활성화 함수를 필요로 할 것이니 시그모이드 함수를 활용하면 될 것이다.

노드가 뭐 10개면 계단 함수로도 될 것 같지만 답은 3이 적당할 듯하다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. XOR 데이터
X = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float32)
Y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float32)

# 2. 모델 정의 (2-2-1 구조 MLP)
class XORNet(nn.Module):
    def __init__(self):
        super(XORNet, self).__init__()
        self.hidden = nn.Linear(2, 2)       # 은닉층: 2 input → 2 output
        self.output = nn.Linear(2, 1)       # 출력층: 2 input → 1 output
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

model = XORNet()

# 3. 손실 함수와 옵티마이저
criterion = nn.BCELoss()                       # Binary Cross Entropy
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 4. 학습 루프
for epoch in range(10000):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, Y)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# 5. 결과 확인
with torch.no_grad():
    output = model(X)
    predicted = (output > 0.5).float()
    print("\n예측 결과:")
    for i in range(4):
        print(f"입력: {X[i].tolist()}, 예측: {predicted[i].item()}, 정답: {Y[i].item()}")


Epoch 0, Loss: 0.7530
Epoch 1000, Loss: 0.6932
Epoch 2000, Loss: 0.6930
Epoch 3000, Loss: 0.6926
Epoch 4000, Loss: 0.6899
Epoch 5000, Loss: 0.6539
Epoch 6000, Loss: 0.5148
Epoch 7000, Loss: 0.4199
Epoch 8000, Loss: 0.3860
Epoch 9000, Loss: 0.3722

예측 결과:
입력: [0.0, 0.0], 예측: 0.0, 정답: 0.0
입력: [0.0, 1.0], 예측: 1.0, 정답: 1.0
입력: [1.0, 0.0], 예측: 0.0, 정답: 1.0
입력: [1.0, 1.0], 예측: 1.0, 정답: 0.0


## **<span style="color:green"> [ 실습 ] </span> 역전파 계산**
코드를 통해 이전에 했던 역전파 계산을 수행 해보겠습니다.


**<span style="color:red"> 1. </span>** 연산에 쓸 모듈과 함수를 정의합니다.

In [ ]:
import math

# 시그모이드 함수와 그 도함수
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

**<span style="color:red"> 2. </span>** 사용할 데이터와, 초기 가중치를 초기화 해줍니다. 강의록에 있던 값을 그대로 쓰되, 편향(b)값을 추가해서 학습해보겠습니다.

In [ ]:
data = [
    ([0.3, 0.2], 0.3)
]

In [ ]:
# 가중치 초기화
W1 = [[0.3, 0.25],  # 은닉층 노드1의 가중치
      [0.5, 0.15]]  # 은닉층 노드2의 가중치
b1 = [0.2, 0.1]     # 은닉층 바이어스

W2 = [0.35, 0.6]     # 출력층 노드의 가중치
b2 = 0.0             # 출력층 바이어스

# 학습률
lr = 0.1

**<span style="color:red"> 3. </span>** 여러번 계산을 하기 위해 forward 연산과 backward연산을 함수로 정의하겠습니다.

In [ ]:
# Forward pass
def forward(x):
    s1 = W1[0][0]*x[0] + W1[0][1]*x[1] + b1[0]  # 첫 번째 노드 계산
    s2 = W1[1][0]*x[0] + W1[1][1]*x[1] + b1[1]  # 두 번째 노드 계산
    h1 = sigmoid(s1)
    h2 = sigmoid(s2)

    s3 = W2[0]*h1 + W2[1]*h2 + b2
    h3 = sigmoid(s3)

    return (s1, s2, h1, h2, s3, h3)

In [ ]:
# Backward pass
def backward(x, y, s1, s2, h1, h2, s3, h3):
    global W1, W2, b1, b2

    # 출력층 오차 및 가중치 업데이트

    dL_dh3 = 2 * (h3 - y)               # MSE 미분
    dh3_ds3 = sigmoid_derivative(h3)    # 시그모이드 미분
    ds3_dW2 = [h1, h2]                  # f(x) = wx + b 를 w에 대해 미분
                                        # 세 값을 모두 곱하면 dL_dW

    # 가중치 업데이트
    for i in range(2):
        W2[i] -= lr * dL_dh3 * dh3_ds3 * ds3_dW2[i]
    b2 -= lr * dL_dh3 * dh3_ds3

    # 은닉층 오차 및 가중치 업데이트
    for i in range(2):
        ds3_dh = W2[i]
        dh_ds = sigmoid_derivative(s1 if i == 0 else s2)
        ds_dw1 = [x[0], x[1]]
        for j in range(2):
            W1[i][j] -= lr * dL_dh3 * dh3_ds3 * ds3_dh * dh_ds * ds_dw1[j]
        b1[i] -= lr * dL_dh3 * dh3_ds3 * ds3_dh * dh_ds

**<span style="color:red"> 4. </span>** 실제로 여러번 학습을 하며 값이 원하는 정답과 가까워 지는 것, 오차가 줄어드는 것을 확인해봅시다.

In [ ]:
# 학습 루프
for epoch in range(100):
    total_loss = 0
    for x, y in data:
        s1, s2, h1, h2, s3, out = forward(x)
        loss = (out - y)**2
        total_loss += loss
        backward(x, y, s1, s2, h1, h2, s3, out)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, 예측: {out:.4f}, Loss: {total_loss:.4f}")

Epoch 0, 예측: 0.6333, Loss: 0.1111
Epoch 10, 예측: 0.5767, Loss: 0.0766
Epoch 20, 예측: 0.5281, Loss: 0.0520
Epoch 30, 예측: 0.4874, Loss: 0.0351
Epoch 40, 예측: 0.4539, Loss: 0.0237
Epoch 50, 예측: 0.4266, Loss: 0.0160
Epoch 60, 예측: 0.4043, Loss: 0.0109
Epoch 70, 예측: 0.3862, Loss: 0.0074
Epoch 80, 예측: 0.3714, Loss: 0.0051
Epoch 90, 예측: 0.3593, Loss: 0.0035


**<span style="color:red">  </span>**  학습을 거듭할수록 예측값이 점점 정답에 가까워 지는 것이 보이시나요???

**<span style="color:red"> 5. </span>** 더 많이 해서 정답이 나올 때 까지 해봅시다

In [ ]:
# 학습 루프
for epoch in range(1000):
    total_loss = 0
    for x, y in data:
        s1, s2, h1, h2, s3, out = forward(x)
        loss = (out - y)**2
        total_loss += loss
        backward(x, y, s1, s2, h1, h2, s3, out)

# 테스트
print("\n=== 테스트 결과 ===")
for x, y in data:
    _, _, _, _, _, out = forward(x)
    print(f"입력: {x}, 예측: {out:.4f}, 정답: {y}")


=== 테스트 결과 ===
입력: [0.3, 0.2], 예측: 0.3000, 정답: 0.3


## 3주차 과제 끝!
3주차 과제는 여기까지 입니다!

수업 중 이해가 안가거나 어려웠던 부분, 수업에 관한 피드백을 적어주시면 다음 주차 강의에 적극적으로 반영하겠습니다!